# Simulating a neural network

Follow the worksheet in order to simulate the use and training of a neural network.
Ignore the following two cells. They contain the implementation of the neural network learner that you will be using in order to do the simulation.

In [ ]:
import numpy as np
import numpy.random as random

from functools import reduce

# The sigmoid function. We use this as an activation function.
# Used for feedforward.
def sigmoid(x):
    return 1.0 / (1 + np.exp(-x))

# The derivative of the sigmoid function. Used for backprop.
def sigmoid_prime(x):
    """"""
    return sigmoid(x) * (1 - sigmoid(x))

def cost(actual, expected):
    """Evaluates the error of a neuron.
    This is the quadratic cost function.
    """
    return 0.5 * (actual - expected) ** 2

def cost_prime(actual, expected):
    """Derivative of the cost function wrt the actual activation.
    The derivative of the quadratic cost function just works out to a difference.
    """
    return actual - expected
        
def scale_gradient(k, g):
    g2 = []
    for layer in g:
        l = []
        for (b, ns) in layer:
            l.append(
                (k * b, [k * x for x in ns])
            )
        g2.append(l)
    return g2
            
def add_gradient(g1, g2):
    """Sums two gradients."""
    g = []
    for layer_l, layer_r in zip(g1, g2):
        n = []
        for (b_l, ns_l), (b_r, ns_r) in zip(layer_l, layer_r):
            n.append(
                (b_l + b_r, [x + y for x, y in zip(ns_l, ns_r)])
            )
        g.append(n)
    return g

class Neuron:
    # A neuron in a NN has:
    # - a bias
    # - a weight for each connection to a neuron in the previous layer as well as one bias. 
    # - an index, specifying which neuron it is in its layer
    
    def __init__(self, j, N, weights=None, bias=None):
        # How many neurons are we connected to in the previous layer?
        self.N = N
        self.index = j
        self.debug_mode = True
        self.prev_layer = None
        self.next_layer = None

        # Initialize the weights randomly, unless weights and bias are provided.
        if weights is not None and bias is not None:
            self.weights = weights
            self.bias = bias
        else:
            self.reset()
        
    def reset(self):
        self.weights = random.randn(self.N) + 0.5 
        self.bias = 0
        
    def debug(self, *args, **kwargs):
        """ Prints a message if debug mode is on. """
        if(self.debug_mode):
            print(*args, **kwargs)
        
    def link_back(self, layer):
        """ Links this neuron to the neurons in the previous layer. """
        assert len(layer) == self.N
        self.prev_layer = layer
        
    def link_forward(self, layer):
        """ Links this neuron to the neurons in the next layers. """
        self.next_layer = layer

    def weighted_input(self, inputs):
        """ Computes the weighted input of this neuron. 
        This is the quantity that goes into the activation function.
        In the literature, this is generally referred to by the letter `z`.
        """
        self.z = np.dot(self.weights, inputs) + self.bias
        return self.z
    
    @property
    def inputs(self):
        if self.prev_layer is None:
            return np.array([])
        return np.array([n.a for n in self.prev_layer])

    def activate(self):
        """Computes the activation of this neuron, and stores it.
        This in turn causes `weighted_input` to be computed and stored.
        """
        if N == 0 and self.a is None:
            raise Error("no activation specified in input neuron")
        self.a = sigmoid(self.weighted_input(self.inputs))
        return self.a
    
    def delta(self, expected=None):
        """Computes the error of this neuron.
        If this is an output neuron, then the delta is computed using the cost function,
        so you must supply an `expected` value.
        Requires that the next layer have its errors already computed.
        """
        if expected is None:
            self.d = sum(
                n.d * n.weights[self.index] * sigmoid_prime(self.z)
                for n
                in self.next_layer
            )
        else:
            self.d = cost_prime(self.a, expected) * sigmoid_prime(self.z)
            
        return self.d
    
class Network:
    def __init__(self, input_layer_size, hidden_layers, output_layer):
        # the network is just a tuple of layers. Each layer is a list of neurons.
        self._network = \
            ([Neuron(j, 0) for j in range(input_layer_size)],) + \
            tuple(hidden_layers) + \
            (output_layer,) # stick output layer on the end
        
        #link each neuron in each layer to its preceding layer
        #the input layer has no preceding layer, so its neurons are not linked to anything.
        for prev_layer, layer in zip(self._network, self._network[1:]):
            for neuron in layer:
                neuron.link_back(prev_layer)
                
        for layer, next_layer in zip(self._network, self._network[1:]):
            for neuron in layer:
                neuron.link_forward(next_layer)
                
    def reset(self):
        for layer in self._network[1:]:
            for neuron in layer:
                neuron.reset()
                
    @property
    def input_layer(self):
        return self._network[0]
              
    @property
    def input(self):
        return tuple(neuron.a for neuron in self.input_layer)
    
    @input.setter
    def input(self, activations):
        assert len(activations) == len(self.input_layer)
        for neuron, a in zip(self.input_layer, activations):
            neuron.a = a
        
    @property
    def output(self):
        return np.array([n.a for n in self._network[-1]])
    
    def feedforward(self, activations):
        """ Computes a prediction for the given input activations. """
        self.input = activations
        for layer in self._network[1:]:
            for neuron in layer:
                neuron.activate()
        return self.output
    
    def backprop(self, expected):
        """ Computes the errors at each neuron for the given expected output vector. """
        input, *layers, out = self._network
        for neuron, y in zip(out, expected):
            neuron.delta(y)
        for layer in reversed(layers):
            for neuron in layer:
                neuron.delta()
                
    @property
    def gradient(self):
        """Computes the gradient of the cost function once all the errors have been calculated.
        The gradient is represented in the same shape as the network.
        Each neuron's portion of the gradient is represented by a tuple with the components:
        1. the partial derivative of the cost function wrt the neuron's bias
        2. the partial derivative of the cost function wrt each of the weights, in order.
        """
        layers = []
        for layer in self._network[1:]:
            neurons = []
            for neuron in layer:
                neurons.append(
                    (neuron.d, [n.a * neuron.d for n in neuron.prev_layer])
                )
            layers.append(neurons)
        return layers
        
    def sgd(self, batch, rate=0.1):
        """Performs stochastic gradient descent on the given batch.
        The batch is a list of pairs of input and expected output.
        """
        gradients = []
        for input, output in batch:
            self.feedforward(input)
            self.backprop(output)
            gradients.append(self.gradient)
        g = reduce(add_gradient, gradients)
        g = scale_gradient(float(rate) / len(batch), g)
        self.apply_gradient(g)
        
    def apply_gradient(self, g):
        for layer, l in zip(self._network[1:], g):
            for neuron, (b, ns) in zip(layer, l):
                neuron.bias -= b
                neuron.weights = np.array(
                    [w - w_ for w, w_ in zip(neuron.weights, ns)]
                )
    
    def evaluate(self, samples):
        correct = 0
        incorrect = 0
        for input, output in samples:
            self.feedforward(input)

In [80]:
# prepare the data
from sklearn.datasets import load_iris
iris = load_iris()

# The targets in the iris dataset are 0, 1, 2
# but our neural network wants a triple of values between zero and one (activations).
# So we make a dictionary that associates *labels* (numbers 0, 1, 2) to an output our NN can understand.

nn_target_map = {
    0: np.array([1, 0, 0]),
    1: np.array([0, 1, 0]),
    2: np.array([0, 0, 1]),
}

nn_targets = []
for t in iris['target']:
    nn_targets.append(
        nn_target_map[t]
    )
# now nn_targets contains appropriate outputs

# samples will be the list of input-output pairs to train the network on
samples = list(zip(iris['data'], nn_targets))

# we also need a way to convert the NN output back into the representation that we started with
# The NN will output a vector of three numbers:
# our conversion will take the index of the greatest value in the vector
def convert_back(a):
    greatest_value = float('-inf')
    greatest_index = -1
    for i, x in enumerate(a):
        if x > greatest_value:
            greatest_value = x
            greatest_index = i
    return greatest_index

# Calculates the accuracy rating of the given network
def evaluate(net, samples):
    correct = 0
    incorrect = 0
    for input, actual in samples:
        actual = convert_back(actual)
        predicted = convert_back(net.feedforward(input))
        if actual == predicted:
            correct += 1
        else:
            incorrect += 1
    return correct / len(samples)
            
def train(net, samples, iterations, mini_batch_size=10, learning_rate=0.1):
    for _ in range(iterations):
        batch = sample(samples, k=mini_batch_size)
        net.sgd(batch, learning_rate)

# Preparing the network

In [ ]:
random.seed(0)

# Prepare the network

# input layer size
N = 4

# first hidden layer
p1 = Neuron(0, N)
p2 = Neuron(1, N)
layer1 = [p1, p2]

# output layer
o1 = Neuron(0, 2)
o2 = Neuron(1, 2)
o3 = Neuron(2, 2)
output = [o1, o2, o3]

# make the network
net = Network(N, [layer1], output)

# Check the initial weights

In [ ]:
p1.weights

In [ ]:
p2.weights

In [ ]:
o1.weights

In [ ]:
o2.weights

In [ ]:
o3.weights

# Do a feedforward

Move on to the next subsection only when every cell in the current subsection has been evaluated by the person responsible for it in your team.

### Set the input

In [129]:
# LEADER:
net.input = samples[0][0]

### Hidden layer activates

In [ ]:
p1.activate()

In [ ]:
p2.activate()

### Output layer activates

In [ ]:
o1.activate()

In [ ]:
o2.activate()

In [ ]:
o3.activate()

### Read the output

In [ ]:
# LEADER:
net.output

In [ ]:
# How does this compare to the expected output?
# Let's look at the expected output:
expected = samples[0][1]
expected

# Do backpropagation to update the network

### Output layer computes errors

The output layer uses the expected values at their layer to determine what their error is.

In [ ]:
o1.delta(expected[0])

In [ ]:
o2.delta(expected[1])

In [ ]:
o3.delta(expected[2])

### Hidden layer computes errors

The hidden layers use the errors of the next layer (in this case the output layer) to determine what their error is. That's why their call to `delta` doesn't take any arguments.

In [ ]:
p1.delta()

In [ ]:
p2.delta()

### Collect the gradient

The gradient represents how sensitive the cost function is changes in each of the parameters (weights or biases) in the network. The gradient is computed from the errors, which is why we needed to calculate and propagate them backwards before we could get to this step.

In [ ]:
# LEADER:
g = net.gradient
g

### Scale the gradient by the learning rate

We want to apply the gradient to the weights and biases to adjust them all so as to make the cost for this training example smaller. But we can't just apply it as is, since it might "overshoot" the optimal parameters.
To rectify this, we scale the gradient by a _learning rate_.

In [ ]:
# LEADER:
learning_rate = 0.1 # shrink each element of the gradient by a factor of 10.
g = scale_gradient(learning_rate, g)
g # confirm that things are smaller by 10x.

### Apply the gradient to the network

Now we can apply the gradient to the network to actually adjust the weights and biases.

In [ ]:
# LEADER:
net.apply_gradient(g)

Do another feed forward (go back to the step labelled "do a feed forward") and see if the output of the network improved!

# Stochastic gradient descent

You may have noticed that the change in output was very slight. That's because we take very very tiny steps along the gradient each time we compute one. Again, we want to avoid taking steps that are too large, or else the optimization of the cost function risks not converging. In practice, this means that on every iteration the parameters change wildly and don't "settle down" on any particular values.

In [ ]:
net.reset()
train(net, samples, iterations=7000) # train the network for 7000 iterations
evaluate(net, samples) # compute the accuracy of the network